In [2]:
# !pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="IAzBWF3Upp02Jo3rTrVq")
project = rf.workspace("ahyun").project("251230_detection")
version = project.version(2)
dataset = version.download("yolov11")
                
                

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
from ultralytics import YOLO

# 1. 모델 로드 (nano 모델 사용)
model = YOLO('yolo11n.pt') 

# 2. 학습 시작
# dataset.location에 있는 data.yaml 파일을 자동으로 참조함
# VS Code 로컬 환경이므로 workers=0 설정이 안정적일 수 있음 (Windows의 경우)
# model.train(
#     data=f"{dataset.location}/data.yaml", 
#     epochs=200,       # 학습 반복 횟수
#     imgsz=640,       # 이미지 크기
#     batch=16,        # 배치 사이즈
#     # patience = 20,
#     plots=True       # 학습 그래프 저장
# )

# 3. 학습 시작
model.train(
    data=f"{dataset.location}/data.yaml",   
    epochs=100,       # 100~200회 권장 (너무 많으면 과적합 가능성)
    imgsz=640,        # 6GB VRAM 기준 640이면 충분함 (OOM 발생 시 512로 줄임)
    batch=16,         # [중요] 6GB VRAM 안전값: 16 (여유 있으면 32 시도 가능)
    device=0,         # GPU 강제 사용 (0번 GPU)
    workers=4,        # 리눅스(우분투)라면 4~8 권장, 윈도우라면 0~2 권장
    project='subway_project', # 결과가 저장될 프로젝트 폴더 이름
    name='train_result',      # 결과가 저장될 실행 이름
    exist_ok=False,    # 덮어쓰기 허용 (False면 train2, train3... 계속 생성됨)
    plots=True        # 학습 결과 그래프 저장
)

New https://pypi.org/project/ultralytics/8.3.243 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.241 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5779MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/juyeong/subway_robot_ws/src/emergency_camera/emergency_camera/251230_detection-2/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a379632aa40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

In [ ]:
from ultralytics import YOLO

# 2. 실험할 파라미터 조합 미리 정의 (List of Dictionaries)
experiments = [
    # 실험 1: 기준 모델 (Baseline)
    {'name': 'exp_baseline', 'batch': 16, 'epochs': 100, 'lr0': 0.01, 'optimizer': 'auto'},
    
    # 실험 2: 배치 사이즈를 키워봄 (속도 및 학습 안정성 테스트)
    {'name': 'exp_batch32',  'batch': 32, 'epochs': 100, 'lr0': 0.01, 'optimizer': 'auto'},
    
    # 실험 3: 학습률을 낮춰봄 (더 정밀하게 학습되는지 테스트)
    {'name': 'exp_low_lr',   'batch': 16, 'epochs': 100, 'lr0': 0.001, 'optimizer': 'AdamW'},
    
    # 실험 4: 이미지 크기를 키움 (작은 객체 탐지 성능 테스트)
    {'name': 'exp_img1024',  'batch': 8,  'epochs': 100, 'lr0': 0.01, 'imgsz': 1024},
]

# 3. 반복문으로 순차적 학습 실행
for config in experiments:
    print(f"\n▶ Starting Experiment: {config['name']}")
    
    # 1. 모델 초기화 (Pretrained 모델 로드)
    model = YOLO('yolo11n.pt')  
    
    # **핵심**: name과 project를 설정해야 결과가 섞이지 않음!
    model.train(
        data=f"{dataset.location}/data.yaml", 
        project='my_yolo_experiments', # 결과가 저장될 상위 폴더 이름
        name=config['name'],           # 각 실험의 폴더 이름
        
        # 가변 파라미터 적용
        batch=config['batch'],
        epochs=config['epochs'],
        lr0=config['lr0'],
        optimizer=config.get('optimizer', 'auto'), # 없으면 기본값 auto
        imgsz=config.get('imgsz', 640),
        
        # patience=20,
        plots=True,
        save=True
    )

print("\n✅ All experiments completed!")